In [16]:
%config IPCompleter.greedy=True
from IPython.display import IFrame

import pandas as pd
import numpy as np
# import string
import math

# import scipy.stats as sts

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from sklearn import preprocessing as prep
# import sklearn.metrics as metrics
import sklearn.model_selection as model_selection
# from sklearn import discriminant_analysis as disan
# from sklearn import calibration as calib
# from sklearn import linear_model as lm
# from sklearn import svm
# from sklearn import gaussian_process as gaup
# from sklearn import mixture as mix
# from sklearn import tree
# from sklearn import ensemble as ens

import tensorflow as tf
from tensorflow import keras

# from keras import models as kermdls
# from keras import layers as kerlrs
# from keras import metrics as kmetrics

# from hyperas import optim
# from hyperas.distributions import choice, uniform
# from hyperopt import Trials, STATUS_OK, tpe

# import pickle

# import nilearn as nl
# from nilearn import plotting, image
# from nilearn import datasets
# import nibabel as nb
import h5py

import os

import time

In [17]:
TEST_IDS = [map_id.split('.')[0] for map_id in sorted(os.listdir('00_Data/fMRI_test'))]
TRAIN_IDS = [map_id.split('.')[0] for map_id in sorted(os.listdir('00_Data/fMRI_train'))]

In [18]:
data = pd.read_csv('00_Data/train_scores_full.csv')
data

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
0,10001,57.436077,30.571975,62.553736,53.325130,51.427998
1,10002,59.580851,50.969456,67.470628,60.651856,58.311361
2,10004,71.413018,53.152498,58.012103,52.418389,62.536641
3,10007,38.617381,49.197021,65.674285,40.151376,34.096421
4,10008,35.326582,15.769168,65.782269,44.643805,50.448485
...,...,...,...,...,...,...
5872,21654,53.103634,50.951656,62.168022,49.389400,53.020847
5873,21665,38.246437,48.018227,59.522285,45.697098,53.208160
5874,21674,69.414169,58.593918,60.298779,49.865669,47.863167
5875,21693,62.009209,54.272484,60.474388,52.325031,52.989803


In [19]:
data.isnull().sum()

Id              0
age             0
domain1_var1    0
domain1_var2    0
domain2_var1    0
domain2_var2    0
dtype: int64

In [20]:
print('Dataset length: ', len(data.index))

Dataset length:  5877


In [21]:
def inputs_gen(idxs, labels):
    for idx, labs in zip(idxs, labels):
        # FNC inputs
        df_fnc = pd.read_csv('00_Data/fnc_csv/{0}.csv'.format(idx), index_col=0)
        X_fnc = np.array(df_fnc.values).reshape(-1)

        # Loading inputs
        df_loading = pd.read_csv('00_Data/loading_csv/{0}.csv'.format(idx), index_col=0)
        X_loading = np.array(df_loading.values).reshape(-1)

        #MRI inputs
        patient_SM = h5py.File('00_Data/fMRI_train/{0}.mat'.format(idx), mode='r')
        patient_SM = np.array(patient_SM.get('SM_feature'))

        k = 2
        ki_padding = 3

        arr_regions = []
        for i in range(patient_SM.shape[0]):
            sample_map = patient_SM[i,:,:,:]
            # padding MRI map
            map_shape = sample_map.shape
            shape_pad = ((map_shape[0]//k + 1)*k - map_shape[0],
                         (map_shape[1]//k + 1)*k - map_shape[1],
                         (map_shape[2]//k + 1)*k - map_shape[2])

            npad = ((shape_pad[0]//2, (shape_pad[0]//2 if shape_pad[0]%2==0 else shape_pad[0]//2+1)),    
                    (shape_pad[1]//2, (shape_pad[1]//2 if shape_pad[1]%2==0 else shape_pad[1]//2+1)),    
                    (shape_pad[2]//2, (shape_pad[2]//2 if shape_pad[2]%2==0 else shape_pad[2]//2+1)))

            sample_map_padded = np.pad(sample_map, pad_width=npad, mode='constant', constant_values=0)

            sx = sample_map_padded.shape[0] / k
            sy = sample_map_padded.shape[1] / k
            sz = sample_map_padded.shape[2] / k
            for kz in range(k):
                for ky in range(k):
                    for kx in range(k):
                        ki_region = sample_map_padded[int(kx*sx): int(kx*sx + sx - 1), 
                                                     int(ky*sy): int(ky*sy + sy - 1), 
                                                     int(kz*sz): int(kz*sz + sz - 1)]
                        # padding i-th region by 3 pixels
                        ki_region_padded = np.pad(ki_region, pad_width=ki_padding, mode='constant', constant_values=0)
                        arr_regions.append(ki_region_padded)
        X_mri = np.stack(arr_regions, axis=3)

    #     X = (X_mri, X_fnc, X_loading)
        yield X_mri, X_fnc, X_loading, labs

In [22]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [23]:
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

In [24]:
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [25]:
def serialize_fullproto_pair(x1, x2, x3, y):
    feat_x1 = _bytes_feature(x1.tostring())
    feat_x2 = _bytes_feature(x2.tostring())
    feat_x3 = _bytes_feature(x3.tostring())
    feat_y = _bytes_feature(y.tostring())
 
    proto_x = tf.train.FeatureList(feature=[feat_x1, feat_x2, feat_x3])
    
    proto_y = feat_y

    sample_protobuf = tf.train.SequenceExample(context=tf.train.Features(feature={'y': proto_y}),
                                               feature_lists=tf.train.FeatureLists(feature_list={'x': proto_x}))
    return sample_protobuf.SerializeToString()

In [32]:
train, test = model_selection.train_test_split(data, test_size=0.2, shuffle=True, random_state=30)
train, val = model_selection.train_test_split(train, test_size=0.2, shuffle=True, random_state=30)
print(len(train.index))

3760


In [49]:
type(train)

pandas.core.frame.DataFrame

In [27]:
tfr_train = 'train.tfrecord'
tfr_val = 'val.tfrecord'
tfr_test = 'test.tfrecord'

ops = tf.io.TFRecordOptions(compression_type="GZIP")

In [30]:
with tf.io.TFRecordWriter(tfr_train, options=ops) as protowriter:
    for i, (x1, x2, x3, y) in enumerate(inputs_gen(train['Id'].values, train[['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']].values)):
        sample = serialize_fullproto_pair(x1, x2, x3, y)
        protowriter.write(sample)
        print("Wrote sample #{}".format(i + 1))
    protowriter.flush()

Wrote sample #1
Wrote sample #2
Wrote sample #3
Wrote sample #4
Wrote sample #5
Wrote sample #6
Wrote sample #7
Wrote sample #8
Wrote sample #9
Wrote sample #10
Wrote sample #11
Wrote sample #12
Wrote sample #13
Wrote sample #14
Wrote sample #15
Wrote sample #16
Wrote sample #17
Wrote sample #18
Wrote sample #19
Wrote sample #20
Wrote sample #21
Wrote sample #22
Wrote sample #23
Wrote sample #24
Wrote sample #25
Wrote sample #26
Wrote sample #27
Wrote sample #28
Wrote sample #29
Wrote sample #30
Wrote sample #31
Wrote sample #32
Wrote sample #33
Wrote sample #34
Wrote sample #35
Wrote sample #36
Wrote sample #37
Wrote sample #38
Wrote sample #39
Wrote sample #40
Wrote sample #41
Wrote sample #42
Wrote sample #43
Wrote sample #44
Wrote sample #45
Wrote sample #46
Wrote sample #47
Wrote sample #48
Wrote sample #49
Wrote sample #50
Wrote sample #51
Wrote sample #52
Wrote sample #53
Wrote sample #54
Wrote sample #55
Wrote sample #56
Wrote sample #57
Wrote sample #58
Wrote sample #59
Wrote 

In [33]:
with tf.io.TFRecordWriter(tfr_val, options=ops) as protowriter:
    for i, (x1, x2, x3, y) in enumerate(inputs_gen(val['Id'].values, val[['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']].values)):
        sample = serialize_fullproto_pair(x1, x2, x3, y)
        protowriter.write(sample)
        print("Wrote sample #{}".format(i + 1))
    protowriter.flush()

Wrote sample #1
Wrote sample #2
Wrote sample #3
Wrote sample #4
Wrote sample #5
Wrote sample #6
Wrote sample #7
Wrote sample #8
Wrote sample #9
Wrote sample #10
Wrote sample #11
Wrote sample #12
Wrote sample #13
Wrote sample #14
Wrote sample #15
Wrote sample #16
Wrote sample #17
Wrote sample #18
Wrote sample #19
Wrote sample #20
Wrote sample #21
Wrote sample #22
Wrote sample #23
Wrote sample #24
Wrote sample #25
Wrote sample #26
Wrote sample #27
Wrote sample #28
Wrote sample #29
Wrote sample #30
Wrote sample #31
Wrote sample #32
Wrote sample #33
Wrote sample #34
Wrote sample #35
Wrote sample #36
Wrote sample #37
Wrote sample #38
Wrote sample #39
Wrote sample #40
Wrote sample #41
Wrote sample #42
Wrote sample #43
Wrote sample #44
Wrote sample #45
Wrote sample #46
Wrote sample #47
Wrote sample #48
Wrote sample #49
Wrote sample #50
Wrote sample #51
Wrote sample #52
Wrote sample #53
Wrote sample #54
Wrote sample #55
Wrote sample #56
Wrote sample #57
Wrote sample #58
Wrote sample #59
Wrote 

In [34]:
with tf.io.TFRecordWriter(tfr_test, options=ops) as protowriter:
    for i, (x1, x2, x3, y) in enumerate(inputs_gen(test['Id'].values, test[['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']].values)):
        sample = serialize_fullproto_pair(x1, x2, x3, y)
        protowriter.write(sample)
        print("Wrote sample #{}".format(i + 1))
    protowriter.flush()

Wrote sample #1
Wrote sample #2
Wrote sample #3
Wrote sample #4
Wrote sample #5
Wrote sample #6
Wrote sample #7
Wrote sample #8
Wrote sample #9
Wrote sample #10
Wrote sample #11
Wrote sample #12
Wrote sample #13
Wrote sample #14
Wrote sample #15
Wrote sample #16
Wrote sample #17
Wrote sample #18
Wrote sample #19
Wrote sample #20
Wrote sample #21
Wrote sample #22
Wrote sample #23
Wrote sample #24
Wrote sample #25
Wrote sample #26
Wrote sample #27
Wrote sample #28
Wrote sample #29
Wrote sample #30
Wrote sample #31
Wrote sample #32
Wrote sample #33
Wrote sample #34
Wrote sample #35
Wrote sample #36
Wrote sample #37
Wrote sample #38
Wrote sample #39
Wrote sample #40
Wrote sample #41
Wrote sample #42
Wrote sample #43
Wrote sample #44
Wrote sample #45
Wrote sample #46
Wrote sample #47
Wrote sample #48
Wrote sample #49
Wrote sample #50
Wrote sample #51
Wrote sample #52
Wrote sample #53
Wrote sample #54
Wrote sample #55
Wrote sample #56
Wrote sample #57
Wrote sample #58
Wrote sample #59
Wrote 

In [59]:
def get_inputs_subm(idxs):
    for idx in idxs:
        # FNC inputs
        df_fnc = pd.read_csv('00_Data/fnc_csv/{0}.csv'.format(idx), index_col=0)
        X_fnc = np.array(df_fnc.values).reshape(-1)

        # Loading inputs
        df_loading = pd.read_csv('00_Data/loading_csv/{0}.csv'.format(idx), index_col=0)
        X_loading = np.array(df_loading.values).reshape(-1)

        #MRI inputs
        patient_SM = h5py.File('00_Data/fMRI_test/{0}.mat'.format(idx), mode='r')
        patient_SM = np.array(patient_SM.get('SM_feature'))

        k = 2
        ki_padding = 3

        arr_regions = []
        for i in range(patient_SM.shape[0]):
            sample_map = patient_SM[i,:,:,:]
            # padding MRI map
            map_shape = sample_map.shape
            shape_pad = ((map_shape[0]//k + 1)*k - map_shape[0],
                         (map_shape[1]//k + 1)*k - map_shape[1],
                         (map_shape[2]//k + 1)*k - map_shape[2])

            npad = ((shape_pad[0]//2, (shape_pad[0]//2 if shape_pad[0]%2==0 else shape_pad[0]//2+1)),    
                    (shape_pad[1]//2, (shape_pad[1]//2 if shape_pad[1]%2==0 else shape_pad[1]//2+1)),    
                    (shape_pad[2]//2, (shape_pad[2]//2 if shape_pad[2]%2==0 else shape_pad[2]//2+1)))

            sample_map_padded = np.pad(sample_map, pad_width=npad, mode='constant', constant_values=0)

            sx = sample_map_padded.shape[0] / k
            sy = sample_map_padded.shape[1] / k
            sz = sample_map_padded.shape[2] / k
            for kz in range(k):
                for ky in range(k):
                    for kx in range(k):
                        ki_region = sample_map_padded[int(kx*sx): int(kx*sx + sx - 1), 
                                                     int(ky*sy): int(ky*sy + sy - 1), 
                                                     int(kz*sz): int(kz*sz + sz - 1)]
                        # padding i-th region by 3 pixels
                        ki_region_padded = np.pad(ki_region, pad_width=ki_padding, mode='constant', constant_values=0)
                        arr_regions.append(ki_region_padded)
        X_mri = np.stack(arr_regions, axis=3)

    #     X = (X_mri, X_fnc, X_loading)
        yield X_mri, X_fnc, X_loading

In [56]:
def serialize_fullproto(x1, x2, x3):
    feat_x1 = _bytes_feature(x1.tostring())
    feat_x2 = _bytes_feature(x2.tostring())
    feat_x3 = _bytes_feature(x3.tostring())

    proto_x = tf.train.FeatureList(feature=[feat_x1, feat_x2, feat_x3])
    
    sample_protobuf = tf.train.SequenceExample(feature_lists=tf.train.FeatureLists(feature_list={'x': proto_x}))
    return sample_protobuf.SerializeToString()

In [57]:
tfr_subm = 'subm.tfrecord'

In [60]:
with tf.io.TFRecordWriter(tfr_subm, options=ops) as protowriter:
    for i, (x1, x2, x3) in enumerate(get_inputs_subm(TEST_IDS)):
        sample = serialize_fullproto(x1, x2, x3)
        protowriter.write(sample)
        print("Wrote sample #{}".format(i + 1))
    protowriter.flush()

Wrote sample #1
Wrote sample #2
Wrote sample #3
Wrote sample #4
Wrote sample #5
Wrote sample #6
Wrote sample #7
Wrote sample #8
Wrote sample #9
Wrote sample #10
Wrote sample #11
Wrote sample #12
Wrote sample #13
Wrote sample #14
Wrote sample #15
Wrote sample #16
Wrote sample #17
Wrote sample #18
Wrote sample #19
Wrote sample #20
Wrote sample #21
Wrote sample #22
Wrote sample #23
Wrote sample #24
Wrote sample #25
Wrote sample #26
Wrote sample #27
Wrote sample #28
Wrote sample #29
Wrote sample #30
Wrote sample #31
Wrote sample #32
Wrote sample #33
Wrote sample #34
Wrote sample #35
Wrote sample #36
Wrote sample #37
Wrote sample #38
Wrote sample #39
Wrote sample #40
Wrote sample #41
Wrote sample #42
Wrote sample #43
Wrote sample #44
Wrote sample #45
Wrote sample #46
Wrote sample #47
Wrote sample #48
Wrote sample #49
Wrote sample #50
Wrote sample #51
Wrote sample #52
Wrote sample #53
Wrote sample #54
Wrote sample #55
Wrote sample #56
Wrote sample #57
Wrote sample #58
Wrote sample #59
Wrote 

KeyboardInterrupt: 

In [7]:
# def new_py_function(func, inp, Tout, name=None):
    
#     def wrapped_func(*flat_inp):
#         reconstructed_inp = tf.nest.pack_sequence_as(inp, flat_inp,
#                                                      expand_composites=True)
#         out = func(*reconstructed_inp)
#         return tf.nest.flatten(out, expand_composites=True)
    
#     flat_Tout = tf.nest.flatten(Tout, expand_composites=True)
#     flat_out = tf.py_function(func=wrapped_func, 
#                               inp=tf.nest.flatten(inp, expand_composites=True),
#                               Tout=[_tensor_spec_to_dtype(v) for v in flat_Tout],
#                               name=name)
#     spec_out = tf.nest.map_structure(_dtype_to_tensor_spec, Tout, expand_composites=True)
#     out = tf.nest.pack_sequence_as(spec_out, flat_out, expand_composites=True)
#     return out

# def _dtype_to_tensor_spec(v):
#     return tf.TensorSpec(None, v) if isinstance(v, tf.dtypes.DType) else v

# def _tensor_spec_to_dtype(v):
#     return v.dtype if isinstance(v, tf.TensorSpec) else v

In [8]:
# def get_dataset(data, batch_size):
#     data = tf.data.Dataset.from_tensor_slices((data['Id'].values, 
#                                                data[['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']].values))
#     data = data.shuffle(buffer_size=6000, seed=30, reshuffle_each_iteration=True)
    
#     data = data.map(lambda idx, lbl:new_py_function(get_inputs, inp=(idx, lbl), 
#                                                     Tout=((tf.TensorSpec(shape=(None, 32,37,32,424), dtype=tf.dtypes.float64, name='input_mri'),
#                                                            tf.TensorSpec(shape=(None, 1378), dtype=tf.dtypes.float64, name='input_fnc'), 
#                                                            tf.TensorSpec(shape=(None, 26), dtype=tf.dtypes.float64, name='input_loading')), 
#                                                           tf.TensorSpec(shape=(None, 5), dtype=tf.dtypes.float64, name='labels')), name=None), 
#                      num_parallel_calls=tf.data.experimental.AUTOTUNE, 
#                      deterministic=True)
#     data = data.batch(batch_size, drop_remainder=False)
#     data = data.prefetch(tf.data.experimental.AUTOTUNE)
#     return data

In [9]:
# train, test = model_selection.train_test_split(data, test_size=0.2, shuffle=True, random_state=30)
# train, val = model_selection.train_test_split(train, test_size=0.2, shuffle=True, random_state=30)

In [10]:
# batch_size = 64

# ds_train = get_dataset(train, batch_size)
# ds_val = get_dataset(val, batch_size)
# ds_test = get_dataset(test, batch_size)

In [1]:
# train_file = 'train.tfrecord'
# ops = tf.io.TFRecordOptions(compression_type="GZIP")
# writer = tf.io.TFRecordWriter(train_file, ops)
# writer.write(ds_train)